In [1]:
import numpy as np
import torch
import glob
import os
import custom_transforms
from tqdm import tqdm
# from data_loader import kitti_training_set, training_data_kitti, test_video
# from model.model2 import Generator_Net, Discriminator_Net, Predict_Net
from data_loader_resize import kitti_training_set, training_data_kitti, test_video
from model.model_resize import Generator_Net, Discriminator_Net, Predict_Net

from torch.utils.data import DataLoader
import time

In [2]:
def tensor2array(tensor):
    tensor = tensor.detach().cpu()
    array = 0.5 + tensor.numpy()*0.5
#     array = array.transpose(1, 2, 0)
    return array

In [3]:
class Config:
        DATASET_PATH = '/media/kuo/32AA7ACBAA7A8ADD/KITTI'
        SINGLE_TEST_PATH = ''
        Pretrained = True
        Pretrained_generator_path = '/media/kuo/32AA7ACBAA7A8ADD/KITTI/VRSA/save_model/generator/generator_6000.tar'
        Pretrained_discriminator_path = '/media/kuo/32AA7ACBAA7A8ADD/KITTI/VRSA/save_model/discriminator/discriminator_6000.tar'
        IS_RGB = False
        BATCH_SIZE = 3

In [5]:
# load dataset
normalize = custom_transforms.Normalize(mean=[0.5], std=[0.5])

train_transform = custom_transforms.Compose([
        custom_transforms.ArrayToTensor(), 
        normalize])

# # kitti_training_set: scenes = ['campus', 'city', 'residential', 'road']
# kitti_training_s = kitti_training_set(DATASET_PATH = Config.DATASET_PATH, scenes=['city', 'residential', 'road'],
#                                         is_rgb = Config.IS_RGB)

# test video
# kitti_training_s = test_video('/media/kuo/32AA7ACBAA7A8ADD/KITTI/VRSA/vrsa_dataset/grayscale/1_1')
kitti_training_s = test_video('/media/kuo/32AA7ACBAA7A8ADD/KITTI/VRSA/vrsa_dataset/img_w:h_2:1/2_3/')

training_data_kitti = training_data_kitti(kitti_training_s, is_rgb = Config.IS_RGB, transform=train_transform)
trainloader_kitti = torch.utils.data.DataLoader(training_data_kitti, batch_size=Config.BATCH_SIZE,shuffle=False)

# generate model
G_net = Generator_Net(is_rgb = Config.IS_RGB).to(torch.device("cuda"))
D_net = Discriminator_Net(is_rgb = Config.IS_RGB).to(torch.device("cuda"))
P_net = Predict_Net(is_rgb = Config.IS_RGB).to(torch.device("cuda"))
if Config.Pretrained:
    G_net.load_state_dict(torch.load(Config.Pretrained_generator_path))
    D_net.load_state_dict(torch.load(Config.Pretrained_discriminator_path))
    print('load success')
else:
    G_net.init_weights()
    D_net.init_weights()
    P_net.init_weights()
    print('initialization success')

load success


In [6]:
# test
scores = []

torch.cuda.synchronize()
start = time.time()

for image in trainloader_kitti:
    imgs = image.cuda()
    generated_image = G_net(imgs)
    score = P_net(torch.abs(imgs-generated_image))
    
torch.cuda.synchronize()
end = time.time()
print(end-start)

276.51896357536316


In [ ]:
249.79
249.24
249.62
249.46
249.95

In [ ]:
250.03
208.13
500.2321934700012
248.98